# Creating World with the Geodynamic World Builder


## Geodynamic World Builder
This notebook describes the workflow for using the [Geodynamic World Builder](https://geodynamicworldbuilder.github.io/) (GWB)  code, a comunity software used to create complex parameterized geometries to represent the Earth's  features such as faults, plumes, and tectonic plates. These features can then be used to described initial conditions, such as temperature or composition, for a geodynamical setting.

### Prerequisites
- Python 3.1 > and its libraries (json, plotly, meshio)
- GWB (version 0.5 or later)
- cmake (version 3.5 or later)

### Installation
To install GWB, run the following commands in a new cell:

```
%cd ../../
! pwd
! mkdir build
%cd build
! pwd
! cmake .. ; make
%cd ../contrib/creating-world-notebook/

```

> Note that the above commands changes the current working directory of the Jupyter notebook so make sure that you are in the parent directory of the module after these commands.

More details on the installation can be found in the [manual here](https://gwb.readthedocs.io/en/v1.0.0/user_manual/installation/stand_alone_install.html)

## Using GWB

To create your world using the GWB, an input text-based JSON file describing the geometry and properties such as temperature, composition etc. is needed. This file, subsequently referred to as the *World Builder (.wb)* file, is enough to create the complicated geometries with specified initial conditions.

To query the generated world at specific points or visualize it on a specified grid, an additional text file usually with a chosen extension of `.dat` and `.grid` with the executable files `WorldBuilder/build/bin/gwb-dat` and `WorldBuilder/build/bin/gwb-grid` , respectively, is needed.

GWB executables are run from the command line. For example, for a given `my_world.wb` file and a `query-points.dat`, use the command:

`WorldBuilder/build/bin/gwb-dat my_world.wb query-points.dat`

>    NOTE : The above line assumes that the executable is in the current working directory. If this is not the case, specify the path to the executable. 

##  Writing the input file 

All the parameters are described in the GWB manual [here](https://gwb.readthedocs.io/en/v1.0.0/GWB_parameter_listings/world_builder_file/index.html). We briefly describe the parameters by loading the world-builder schema file into a variable, `gwb_schema`.

1. **Global header information** : There are several parameters that describe the global parameters which will be used while creating the World, such as the `version`, `coordinate system` etc. To check the names and description of all of these parameters, you can access the `properties` item stored in the world-builder schema file and the structure for each of the `properties` is defined using `$schema` as `gwb_schema['properties']['$schema']`.

2.  **Features** : GWB supports several different feature models corresponding to a [tectonic feature](https://gwb.readthedocs.io/en/v1.0.0/user_manual/parameter_documentation/features/index.html). To check the available features, you can run `gwb_schema['properties']['features']['items']['defaultSnippets']` in this notebook. More information about each of the available features can be found using `gwb_schema['properties']['features']['items']['oneOf'][$num]`, where `$num` is an integer corresponding to the index in the order of the list of models obtained from the previous command. 

The geometry of the feature can vary laterally as well as along the down-dip direction for the `fault` and the `subducting plate` models by adding `sections` and `segments`, respectively. *Note that all 'sections' should have the same number of the 'segments'.*


Each of these features and their segments and sections can have different properties, such as temperature, composition, grains, velocity. The values of these prescribed properties can be `uniform` or follow some other available distribution for that property. You can get more information on the available property distributions using the schema file. 

For example, `gwb_schema['properties']['features']['items']['oneOf'][1]['properties']['composition models']['items']['oneOf']` outputs the list of available composition model for a fault (fault is second in the list of available feature models, therefore we use index 1 to output its properties).

In [1]:
import subprocess
import meshio
from glob import glob
import plotly.graph_objects as go
from scipy.interpolate import griddata
import numpy as np

import json
from jsonschema import validate

In [2]:
# The `.wb` file follows a json schema describing the global parameters and the model features.
# The schema can be used to investigate the available model features how to describe them 
# in GWB.

schema_file = open('../../doc/world_builder_declarations.schema.json')
gwb_schema  = json.load(schema_file)

## Playtime : Creating continents 

In this module, we will create continents with their traces following the letters "GWB". This example is chosen just to demonstrate how to incorporate a complex geometry using GWB.

In [3]:
# STEP 1: First let's create the skeleton file that includes the global header information 
# and the model features. 

wb_file = { "version": "1.1",
            "interpolation": "continuous monotone spline",
            "coordinate system":{"model":"cartesian"},
            "features": []
          }

In [4]:
def make_plate_feature(plate_name, coordinates, min_depth=-1, max_depth=10):
    """
    This function takes an input fault names and its trace coordinates to create a 
    fault feature model for the WB file. 
    Several parameters to describe the fault are set to default values for now.

    Parameters
    ----------
    fault_name : str
        The name we want to give to the fault.
    coordinates : list
        The coordinates of the fault trace as list of [x, y] points.
        
    Returns
    -------
    out : dict
        The dictionary containing the fault feature model.
    """
    
    out = {
    "model": "continental plate",
    "name": plate_name,
    "min depth": min_depth,
    "max depth": max_depth,
    "coordinates": coordinates,
        
    "composition models": [{"model":"uniform", "compositions":[0], "max depth": 10}] 
     }
    
    return out

In [5]:
# STEP 2 :  Load the accompanying file that contains plate coordinates such that 
# each enclosed plate is separated by a "<" character. 
# Store all the feature models as a list of dictionaries, as needed by the GWB

input_file_name = 'plates.csv'
all_plates_json = []

# Reading the content of the file
with open(input_file_name, 'r') as file:
    lines = file.read().split("<")

# Determine the number of plates (assuming each block between "<" corresponds to one plate)
number_of_plates = len(lines)

for i in range (1, int(number_of_plates) - 1):
    coordinates = lines[i].strip().split()

    # Convert the list of string coordinates to floats
    list_of_floats = [float(item) for item in coordinates]

    # Split into x and y coordinates
    coordinates_x = list_of_floats[0::2]
    coordinates_y = list_of_floats[1::2]

    # Combine into a coordinate array
    coordinates_array = list(zip(coordinates_x, coordinates_y))

    # Append the result
    all_plates_json.append(make_plate_feature('plate_' + str(i), coordinates_array))

In [6]:
# STEP 3: Create the json file that includes all the continental plates 

output_wb_file = "my_world_builder.wb"
wb_file['features'] = all_plates_json

with open(output_wb_file, "w") as f :
    json.dump(wb_file, f)

In [7]:
# STEP 4: Run the GWB using the .wb file generated above using the
# accompanying .grid file, 'cartesian_surface.grid'
# This .grid file is a simple text file that contains the extent and
# the discretization of the grid. We need to make sure that the extent 
# of the grid is big enough to include all the plates.

gwb_grid_location = "../../build/bin/gwb-grid "
grid_file_name    = " cartesian_surface.grid"
command_to_run    = gwb_grid_location + output_wb_file + grid_file_name

subprocess.run(command_to_run, shell=True)

CompletedProcess(args='../../build/bin/gwb-grid my_world_builder.wb cartesian_surface.grid', returncode=0)

## Visualizing the results

### Meshio
To visualize the generated world, we use the `gwb-grid` to query the properties on a grid file, `cartesian_surface.grid` available as one of the [tests](https://github.com/GeodynamicWorldBuilder/WorldBuilder/tree/main/tests) in GWB.

In [8]:
def read_vtu_file (file_name):
    """
    This function takes an input .vtu file_name (either structured or
    unstructured mesh) and returns the point coordinates and values in that file.  
    
    Parameters
    ----------
    file_name : str
        The name of the vtu file
        
    Returns
    -------
    nodes  : A numpy array with columns corresponding to the 
             coordinates of the node points (x, y, z) in the input mesh.
    
    data_out : A dictionary containing key:values corresponding to the 
               available fields and their values in the input .vtu file.
    """
    
    mesh     = meshio.read(file_name)
    nodes    = mesh.points
    data_out = mesh.point_data
    
    return (nodes, data_out)

In [9]:
vtu_file = output_wb_file.replace('.wb', '.vtu')
coordinates, data = read_vtu_file (vtu_file)

In [10]:
%%capture 

# Create the 3D scatter plot using Plotly
points_x = coordinates[:, 0]
points_y = coordinates[:, 1]
points_z = coordinates[:, 2]

x_ = np.linspace(min(points_x), max(points_x), 80)
y_ = np.linspace(min(points_y), max(points_y), 40)
z_ = np.linspace(min(points_z), max(points_z), 5)

values = data['Composition 0']

X, Y, Z = np.meshgrid(x_, y_, z_, indexing='ij')
values   = griddata((points_x, points_y, points_z), values, (X, Y, Z), method='nearest')

# Create a meshgrid with the spherical coordinates

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(), 
    colorscale='rdbu_r', 
    colorbar = dict(title='Composition', tickvals=[0, 1], ticktext=['0', '1']),
    opacity=0.2,
    surface_count=30,
    ))

camera = dict( eye=dict(x=0, y=0, z=4),  
              up=dict(x=0, y=1, z=0),     
              center=dict(x=0, y=0, z=0))

fig.update_scenes(aspectmode='data')
fig.update_layout(scene_camera=camera, title="GWB Output")

### Paraview Glance
Another tool to visualize the models is to use the [Glance](https://kitware.github.io/glance/app/) app. You can easily load the generated `.vtu` file from the GWB in Glance and rotate around the modeled results and modify the visualization field.

### Exercise: things to try

* Change the max depth 
* Change composition value to 1 and modify the current .grid file for 2 compositions
* Plot temperatures instead of Composition
* Add more features
* Modify the resolution by changing, one of more of n_cell_x, n_cell_y, n_cell_z, in the.grid file